In [2]:
import pandas as pd

In [42]:
string_links_fi = "/mnt/home/ssledzieski/database/stringdb/dmelanogaster/7227.protein.physical.links.v12.0.txt"
biogrid_links_fi = "/mnt/home/ssledzieski/database/bioGRID/BIOGRID-ORGANISM-Drosophila_melanogaster-4.4.240.tab3.txt"
flybase_id_map_fi = (
    "/mnt/home/ssledzieski/database/flybase/fbgn_NAseq_Uniprot_fb_2024_06.tsv"
)
flybase_gn_pp_fi = (
    "/mnt/home/ssledzieski/database/flybase/fbgn_fbtr_fbpp_fb_2024_04.tsv"
)

predictions_fi = "/mnt/home/ssledzieski/Projects/philharmonic-experiments/results/d_melanogaster/20240913-dmelaAll-phil_v0.2.0/20240912-dmelaAll-phil_v0.2.0_network.tsv"
predictions_pos_fi = "/mnt/home/ssledzieski/Projects/philharmonic-experiments/results/d_melanogaster/20240913-dmelaAll-phil_v0.2.0/20240912-dmelaAll-phil_v0.2.0_network.positive.tsv"

In [ ]:
string_links = pd.read_csv(string_links_fi, sep=" ")
biogrid_links = pd.read_csv(biogrid_links_fi, sep="\t", header=0)
flybase_id_map = pd.read_csv(flybase_id_map_fi, sep="\t", header=None, comment="#")
flybase_id_map.columns = [
    "gene_symbol",
    "organism",
    "FBgn",
    "accession",
    "na_accession",
    "uniprot",
    "entrez",
    "refseq_transcript",
    "refseq_protein",
]
flybase_gn_pp = pd.read_csv(
    flybase_gn_pp_fi, sep="\t", header=None, comment="#"
).dropna()

predictions = pd.read_csv(predictions_fi, sep="\t", header=None)
predictions_pos = pd.read_csv(predictions_pos_fi, sep="\t", header=None)

In [88]:
def alphabetize(df, colA, colB):
    # lexically sort the two columns so that the smaller value is always in the first column
    df[[colA, colB]] = df[[colA, colB]].apply(sorted, axis=1, result_type="expand")
    return df

In [84]:
import numpy as np

flybase_gn_pp_dict = {row[0]: row[2] for row in flybase_gn_pp.itertuples(index=False)}


def strip_map(x):
    if x in flybase_gn_pp_dict:
        k = flybase_gn_pp_dict[x]
        # print(k, type(k))
        if isinstance(k, float) and np.isnan(k):
            return None
        return f"7227.{k}"
    return None

In [85]:
flybase_id_map["FBpp"] = flybase_id_map["FBgn"].map(strip_map)

In [86]:
flybase_id_map[~flybase_id_map["FBpp"].isna()]

,gene_symbol,organism,FBgn,accession,na_accession,uniprot,entrez,refseq_transcript,refseq_protein,FBpp
20,a,Dmel,FBgn0000008,AE013599,AAF46809,NaN,NaN,NaN,NaN,7227.FBpp0309745
21,a,Dmel,FBgn0000008,AE013599,AAF46810,NaN,NaN,NaN,NaN,7227.FBpp0309745
22,a,Dmel,FBgn0000008,AE013599,AAX52678,NaN,NaN,NaN,NaN,7227.FBpp0309745
23,a,Dmel,FBgn0000008,AF188473,AAF37816,NaN,NaN,NaN,NaN,7227.FBpp0309745
24,a,Dmel,FBgn0000008,AF188474,AAF37817,NaN,NaN,NaN,NaN,7227.FBpp0309745
...,...,...,...,...,...,...,...,...,...,...
937129,Hsp27,Dmel,FBgn0289994,NaN,NaN,X2JC82,NaN,NaN,NaN,7227.FBpp0312159
937130,Hsp27,Dmel,FBgn0289994,NaN,NaN,P02518,NaN,NaN,NaN,7227.FBpp0312159
937131,Hsp27,Dmel,FBgn0289994,NaN,NaN,NaN,39078.0,NaN,NaN,7227.FBpp0312159
937132,Hsp27,Dmel,FBgn0289994,NaN,NaN,NaN,NaN,NM_079276,NP_524000,7227.FBpp0312159


In [104]:
predictions_pos.shape

(197510, 3)

In [106]:
string_links.shape

(333606, 3)

In [107]:
string_alpha = alphabetize(string_links, "protein1", "protein2")
pred_alpha = alphabetize(predictions, 0, 1)
pred_positive = pred_alpha[pred_alpha[2] >= 0.5]

In [109]:
pred_positive.head()

,0,1,2
5614,7227.FBpp0082477,7227.FBpp0289929,0.895468
5998,7227.FBpp0079704,7227.FBpp0289929,0.773858
6235,7227.FBpp0077303,7227.FBpp0289929,0.903024
6811,7227.FBpp0073477,7227.FBpp0289929,0.816850
6909,7227.FBpp0080733,7227.FBpp0289929,0.908572


In [102]:
merged = pd.merge(
    string_alpha,
    pred_alpha,
    left_on=["protein1", "protein2"],
    right_on=[0, 1],
    how="outer",
)

In [103]:
merged

,protein1,protein2,combined_score,0,1,2
0,7227.FBpp0070001,7227.FBpp0100140,292.0,NaN,NaN,NaN
1,7227.FBpp0070001,7227.FBpp0100140,292.0,NaN,NaN,NaN
2,7227.FBpp0070001,7227.FBpp0302615,183.0,NaN,NaN,NaN
3,7227.FBpp0070001,7227.FBpp0302615,183.0,NaN,NaN,NaN
4,7227.FBpp0070002,7227.FBpp0070006,NaN,7227.FBpp0070002,7227.FBpp0070006,0.004235
...,...,...,...,...,...,...
15531227,7227.FBpp0423136,7227.FBpp0423170,175.0,NaN,NaN,NaN
15531228,7227.FBpp0423146,7227.FBpp0423186,322.0,NaN,NaN,NaN
15531229,7227.FBpp0423146,7227.FBpp0423186,322.0,NaN,NaN,NaN
15531230,7227.FBpp0423205,7227.FBpp0423326,191.0,NaN,NaN,NaN


In [101]:
merged[merged[2] > 0.05]

,protein1,protein2,combined_score,0,1,2
0,7227.FBpp0070006,7227.FBpp0075118,420,7227.FBpp0070006,7227.FBpp0075118,0.950745
2,7227.FBpp0070006,7227.FBpp0070351,541,7227.FBpp0070006,7227.FBpp0070351,0.971481
10,7227.FBpp0070031,7227.FBpp0072179,218,7227.FBpp0070031,7227.FBpp0072179,0.825208
11,7227.FBpp0070031,7227.FBpp0074978,218,7227.FBpp0070031,7227.FBpp0074978,0.966129
12,7227.FBpp0070031,7227.FBpp0293897,155,7227.FBpp0070031,7227.FBpp0293897,0.978932
...,...,...,...,...,...,...
25472,7227.FBpp0081725,7227.FBpp0305898,499,7227.FBpp0081725,7227.FBpp0305898,0.962937
25475,7227.FBpp0086694,7227.FBpp0306001,150,7227.FBpp0086694,7227.FBpp0306001,0.065004
25479,7227.FBpp0075488,7227.FBpp0306007,420,7227.FBpp0075488,7227.FBpp0306007,0.487673
25481,7227.FBpp0076872,7227.FBpp0306023,612,7227.FBpp0076872,7227.FBpp0306023,0.600007
